# Import Packages

In [1]:
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.optimizers import Adam

import numpy as np

import os
import matplotlib.pyplot as plt

from tqdm import tqdm
from google.colab import files


Using TensorFlow backend.


# Model Variables

In [0]:
imageRows = 28
imageColumns = 28  
imageChannels = 1

imageShape = (imageRows, imageColumns, imageChannels)

imageLatentDimension = 100

optimizer = Adam(0.0002, 0.5)

# Models 

**Generator Model**

In [0]:
def generator():
  
  model = Sequential()
  
  model.add(Dense(128 * 7 * 7, activation = 'relu', input_dim = imageLatentDimension))
  model.add(Reshape((7, 7, 128)))
  
  model.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.01))
  
  model.add(Conv2DTranspose(64, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.01))
  
  model.add(Conv2DTranspose(32, kernel_size = 3, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.01))
  
  model.add(Conv2D(imageChannels, kernel_size = 3, padding = "same"))
  model.add(Activation("tanh"))
  
  model.summary()
  
  randomNoise = Input(shape = (imageLatentDimension, ))
  generatedImage = model(randomNoise)

  return Model(inputs = randomNoise, outputs = generatedImage)

**Discriminator Model**

In [0]:
def discriminator():

  model = Sequential()

  model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = imageShape, padding = "same"))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(BatchNormalization(momentum = 0.8))

  
  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = "same"))
  model.add(ZeroPadding2D(padding = ((0, 1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  
  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = "same"))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  
  model.add(Conv2D(256, kernel_size = 3, strides = 1, padding = "same"))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  
  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))

  model.summary()

  inputImage = Input(shape = imageShape)
  validity = model(inputImage)

  return Model(inputs = inputImage, outputs = validity)

# Driver Code

In [5]:
# Visualise Models
print ("Discriminator Model")
discriminatorModel = discriminator()

print ("\n\nGenerator Model")
generatorModel = generator()

# Compile Discrimiator Model
discriminatorModel.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Generator Takes Random Noise As Input, Ergo Create Placeholder
randomNoise = Input(shape=(imageLatentDimension, ))
generatedImage = generatorModel(randomNoise)

# For The Final Combined Model, Train Only Generator
discriminatorModel.trainable = False

# Discriminator Takes Generated Images And Computes Validity
validity = discriminatorModel(generatedImage)

# Combined Model
combinedModel = Model(inputs = randomNoise, outputs = validity)
combinedModel.compile(optimizer = optimizer, loss = 'binary_crossentropy')

Discriminator Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0

# Train 

In [0]:
def train(epochs, batchSize = 128, saveInterval = 50):

  # Shape : (60000, 28, 28) 
  (X_train, _), (_, _) = mnist.load_data()

  # Rescale -1 To 1
  X_train = X_train / 127.5 - 1.
  
  # Expand Final Dimension To Include Channel (Shape Is Now (60000, 28, 28, 1))
  X_train = np.expand_dims(X_train, axis = 3)
  
  # Ground Truths For Adversarial Network
  real = np.ones((batchSize, 1))
  fake = np.zeros((batchSize, 1))
  
  for epoch in tqdm(range(epochs)):
    
      # ----------------------------------

      #         Train Discriminator

      # ----------------------------------
    
      # Select Random Batch Of Images
      imageIndices = np.random.randint(0, X_train.shape[0], batchSize)
      imagesToTrain = X_train[imageIndices]

      # Sample Noise And Generate Images
      randomNoise = np.random.normal(0, 1, (batchSize, imageLatentDimension))
      generatedImages = generatorModel.predict(randomNoise)
      
      # Train Discriminator By Classifying Real As 1's and Generated As 0's
      discriminatorLossReal = discriminatorModel.train_on_batch(imagesToTrain, real)
      discriminatorLossFake = discriminatorModel.train_on_batch(generatedImages, fake)
      discriminatorLoss = 0.5 * np.add(discriminatorLossReal, discriminatorLossFake)
    
      # ----------------------------------

      #         Train Generator

      # ----------------------------------

    
      generatorLoss = combinedModel.train_on_batch(randomNoise, real)
      
      # Print Progress
      print ("%d [D Loss: %f, Accuracy: %.2f%%] [G Loss: %f]" % (epoch, discriminatorLoss[0], 100 * discriminatorLoss[1], generatorLoss))
  
      # Save Images
      if epoch % saveInterval == 0:

        saveImages(epoch)
  


In [0]:
def saveImages(epoch):
       
        r, c = 5, 5
        randomNoise = np.random.normal(0, 1, (r * c, imageLatentDimension))
        generatedImages = generatorModel.predict(randomNoise)

        # Rescale Images From 0 - 1
        generatedImages = 0.5 * generatedImages + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(generatedImages[cnt, :, :, 0], cmap = 'gray')
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("mnist_model%d.png" % epoch)
        plt.close()


In [0]:
train(epochs = 2000, batchSize = 32, saveInterval = 500)

11493376/11490434 [==============================] - 1s 0us/step


  0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D Loss: 1.277752, Accuracy: 25.00%] [G Loss: 0.731562]


  0%|          | 2/2000 [00:06<2:30:47,  4.53s/it]

1 [D Loss: 0.418644, Accuracy: 78.12%] [G Loss: 1.812042]


...

100 [D Loss: 0.479345, Accuracy: 78.12%] [G Loss: 2.927656]


  5%|▌         | 102/2000 [02:01<51:23,  1.62s/it]

101 [D Loss: 0.678319, Accuracy: 57.81%] [G Loss: 2.514822]


  5%|▌         | 103/2000 [02:02<51:18,  1.62s/it]

102 [D Loss: 0.386325, Accuracy: 81.25%] [G Loss: 3.005407]


In [1]:
for number in range(0, 4000, 100):
  file = 'mnist_model' + str(number) + '.png'
  
  try :
    files.download(file)
  except :
    None
    
print ("Images Downloaded To Local Drive")

Images Downloaded To Local Drive
